In [ ]:
%load_ext autoreload
%autoreload 2
from IPython import get_ipython; ipython = get_ipython()
#%run -i ../exec_HPC.py 0
import os 
import sys
from os.path import join as pjoin
p = pjoin( os.path.expandvars('$CODE_MEMORY_ERRORS'), 'state-space-adaptation' )
sys.path.append(p)
sys.path.append(os.path.expandvars('$CODE_MEMORY_ERRORS'))
import pandas as pd
import time
import numpy as np
from config2 import path_data

import state_space_Tan2014 as tan
from state_space_Tan2014 import *
import matplotlib.pyplot as plt

fnf_NIH = pjoin(path_data,'df_all_multi_tsz_.pkl.zip')
dfcc_all_NIH_ = pd.read_pickle(fnf_NIH)
dfcc_all_NIH = dfcc_all_NIH_.query('trial_shift_size == 1 and '
    'trial_group_col_calc == "trials" and retention_factor_s == "0.924"').copy()
assert not dfcc_all_NIH.duplicated(['subject','trials']).any()
from datetime import datetime
print('Data mtime',datetime.fromtimestamp(os.stat(fnf_NIH).st_mtime))

subjects_NIH = dfcc_all_NIH['subject'].unique()
from taumodels.models import fitDiedrischenModel,fitTanModel

# # main run
# nt = 1000
# nsub = 4
# subjects_sub = subjects[0:nsub]
# numtrain = 12
# dfc = df.query('subject in @subjects_sub and trial_index < @nt and trial_index >= @numtrain').copy()
# dfc.loc[dfc.query('error_lh2_ang_deg.abs() > 60').index, 'error_lh2_ang_deg'] = np.nan
# # TODO: maybe I should invalidate too big errors, they are clearly outliers
n_jobs = 8
numtrain = 12
# do_plot = 0
# #ipython.run_line_magic('run',' -i ' + scr)

inds = dfcc_all_NIH.query('environment == 1').index
dfcc_all_NIH.loc[inds,'perturbation'] = dfcc_all_NIH.loc[inds,'feedback'] - dfcc_all_NIH.loc[inds,'org_feedback']
dfcc_all_NIH.loc[inds,'perturbation'] *= 180 / np.pi
#[]
dfcc_all_NIH['trial_index'] = dfcc_all_NIH['trials']

path_data_cc = '/home/demitau/data_Quentin/full_experiments/context_change_behav/results'

dfcc_all_NIH['error_deg'] = dfcc_all_NIH['error'] * 180 / np.pi

coln_error = 'error_lh2_ang_deg'

In [ ]:
dfcc_all_NIH.query('subject == @subjects_NIH[0]').\
    to_pickle(pjoin(path_data,'df_all_ext_onesubj.pkl.zip'),
              compression='zip')

In [ ]:
from itertools import product
#stats_powers_all = [(2,-2),(1,-1),(1,0),(2,0),(0,-1),(0,-2),(1,-2),(2,-1)]
stats_powers_all = [(0,-1),(0,-2),(1,-2)]
prod = product(stats_powers_all,[0,1],[0,1],[0.,1.5,3.,6],['all','stable','random'])
prod = list(prod)
prod

# Tan

In [ ]:
dfnamesubj2res_Tan = {}
nruns = 20
n_jobs = 32
#reg = 1e-3
#reg = 10
#reg = 5
#reg = 1
#reg = 3
nruns = 5
#nsubj = 20
minStartIdx =3; maxStartIdx=7; maxNbTrials = 11; minNbTrials = 2
#stats_powers = (2,-2)
#stats_powers = (0,-1)
#fit_trials = 'stable' # the problem is how to reset state between to blocks
#use_true_error=0; use_true_error_stats=1
#nsubj=1; nruns=3; reg=3; fit_trials='stable'; minStartIdx =6; maxStartIdx=7; maxNbTrials = 7;minNbTrials = 2
nsubj=3; nruns=4; minStartIdx =6; maxStartIdx=7; maxNbTrials = 6;minNbTrials = 3
#for dfname,df in zip(['NIH', 'cc'], [dfcc_all_NIH,dfcc_all]):
for dfname,df in zip(['NIH'], [dfcc_all_NIH,dfcc_all]):
    for subj in df.subject.unique()[:nsubj]:
        print(dfname, '   subj == ',subj)
        qs0 = 'subject == @subj'
        
        motor_var = df.query(qs0 + ' and trial_index >= 10 and trial_index <= 24')['error'].abs().std()
        t0 = time.time()
        inds_state_reset = []  
        
        dfnamesubj2res_Tan[(dfname,subj)]= []
        for (stats_powers,use_true_error,use_true_error_stats,reg,fit_trials) in prod:
            if dfname == 'NIH':
                coln_error_ = 'error_deg'                       
                if fit_trials == 'stable':
                    qs = qs0 + ' and environment == 0'
                    #inds_state_reset = np.arange(192,700, 192)
                    inds_state_reset = np.array([192])
                elif fit_trials == 'random':
                    qs = qs0 + ' and environment == 1'
                    #inds_state_reset = np.arange(192,700, 192)
                    inds_state_reset = np.array([192])
                else:
                    qs = qs0
            else:         
                qs = qs0 + ' and trial_index >= @numtrain'            
                coln_error_ = coln_error
            dfos = df.query(qs)    
            
            
            motor_var_ = motor_var
            if use_true_error:
                motor_var_ = 0. # would just confuse model additionally
            
            out_cursubj = None
            import warnings
            with warnings.catch_warnings(record=True) as w:
                warnings.filterwarnings('ignore',category=RuntimeWarning)

                out_cursubj = fitTanModel(dfos, coln_error_, n_jobs, 
                            bclip=(-1,1), nruns=nruns,
                         reg = reg, neg_error=1,
                        maxNbTrials = maxNbTrials,
                        minNbTrials = minNbTrials,
                        minStartIdx = minStartIdx,
                        maxStartIdx=maxStartIdx, cmax = 0.007, online_feedback=False, 
                        inds_state_reset = inds_state_reset, stats_powers=stats_powers,
                        motor_var = motor_var_,
                        use_true_error=use_true_error, use_true_error_stats=use_true_error_stats)
        
            out_cursubj['fit_trials'] = fit_trials
            dfnamesubj2res_Tan[(dfname,subj)].append( out_cursubj )
            #for subj in subj2out_Tan: 
            #    print('     ',subj)
            #    for k,v in subj2out_Tan[subj ].items():
            print(out_cursubj['params'],stats_powers,use_true_error,use_true_error_stats )

            perturb = dfos['perturbation'].to_numpy()    
            adaptationRate = out_cursubj['adaptation_rate']
            output = out_cursubj['y_pred']
            error = dfos['error_deg'].values

            plot = 0
            if plot:
                plt.figure(figsize=(15,3)) # Use matplotlib.pyplot to plot figures
                ttl = 'Subject {}, optres={}, pows={} onlfb={}, truerr={}, trueerrst={}'.format( 
                    dfos['subject'].values[0][:5],
                    out_cursubj['optres']['success'],out_cursubj['stats_powers'], 
                        out_cursubj['online_feedback'],out_cursubj['use_true_error'], 
                        out_cursubj['use_true_error_stats'] )            
                plt.title(ttl) # Use string concatenation to make titles
                plt.plot(error, linewidth=2, label='Observed error') # Use label argument to add legends
                plt.plot(output, linewidth=2, label='Model prediction',alpha=0.8)
                plt.ylabel('Angle (°)')
                plt.xlabel('Trials')
                plt.plot(perturb, label='pert', color='red', ls=':')
                plt.legend() # Call legend function to show legends
                #plt.twinx() # Use twinx function to create a second y-axis
                if adaptationRate is not None:
                    plt.plot(30 + 50*adaptationRate, linewidth=1, label='Adaptation rate', color='green')            
                    plt.legend() # Call legend function to show legends
                plt.ylim(-40,60)
                plt.grid(True)
                plt.show() # Call

In [ ]:
import gc; gc.collect()

In [ ]:
len( list(dfnamesubj2res_Tan.values())[0] )

In [ ]:
out_cursubj.keys()

In [ ]:
out_cursubj['minfval']

In [ ]:
ttl = 'Subject {}, optres={}, pows={} onlfb={}, truerr={}, trueerrst={}'.format( 
            dfos['subject'].values[0][:5],
            out_cursubj['optres']['success'],
                                  out_cursubj['stats_powers'], 
                out_cursubj['online_feedback'],out_cursubj['use_true_error'], 
                out_cursubj['use_true_error_stats'] )
ttl

In [ ]:
import gc; gc.collect()

In [ ]:
len(dfnamesubj2res_Tan)

In [ ]:
['minres'].success

In [ ]:
len(dfnamesubj2res_Tan)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
out['params'].keys()

In [ ]:
def out2parstr(out):
    output = []
    # Loop through the keys and values of the dictionary
    for key, value in out['params'].items():
        # If the value is a float, use the formatting with 3 digits after the decimal point
        if isinstance(value, float):
            if key == 'c':
                output.append(f"{key}={value:g}")
            else:
                output.append(f"{key}={value:.2f}")
        # Otherwise, append the value without formatting
        else:
            output.append(f"{key}={value}")

    # Join the list of strings with a newline character as a separator
    output = '; '.join(output)
    return output

In [ ]:
out.keys()

In [ ]:
# recompute 1
out = outs[0]
ttl = 'Subject {}, optres={}, pows={} onlfb={}, truerr={}, trueerrst={}, reg={}, minf={:.3f}'.format( 
        dfos['subject'].values[0][:5],
        out['optres']['success'],
            out['stats_powers'], 
            out['online_feedback'],out['use_true_error'], 
            out['use_true_error_stats'], out['reg'], out['minfval'] )
print(ttl,'\n',out2parstr(out))
#reg = 3
reg = 0.
out = fitTanModel(dfos, coln_error_, n_jobs, 
                bclip=(-1,1), nruns=nruns,
            reg = reg, neg_error=1,
            maxNbTrials = 14,minNbTrials = 2,
            minStartIdx = 2, maxStartIdx=10, online_feedback=False, 
            cmax = 0.006,
            inds_state_reset = inds_state_reset, stats_powers=(2,-2),
            use_true_error=0, use_true_error_stats=1)

plot = 1
if plot:
    adaptationRate = out['adaptation_rate']
    plt.figure(figsize=(15,3)) # Use matplotlib.pyplot to plot figures
    plt.title('Subject ' + dfos['subject'].values[0]) # Use string concatenation to make titles
    plt.plot(error, linewidth=2, label='Observed error') # Use label argument to add legends
    plt.plot(out['y_pred'], linewidth=2, label='Model prediction',alpha=0.8)
    plt.plot(out['states'], label='states', c='purple')
    plt.ylabel('Angle (°)')
    plt.xlabel('Trials')
    plt.plot(perturb, label='pert', color='red', ls=':')
    plt.legend() # Call legend function to show legends
    #plt.twinx() # Use twinx function to create a second y-axis
    if adaptationRate is not None:
        plt.plot(30 + 50*adaptationRate, linewidth=1, label='Adaptation rate', color='green')            
        plt.legend() # Call legend function to show legends
    plt.ylim(-40,60)
    plt.grid(True)

    ttl = 'Subject {}, optres={}, pows={} onlfb={}, truerr={}, trueerrst={}, reg={}, minf={:.3f}'.format( 
            dfos['subject'].values[0][:5],
            out['optres']['success'],
                out['stats_powers'], 
                out['online_feedback'],out['use_true_error'], 
                out['use_true_error_stats'], out['reg'], out['minfval'] )
    plt.title(ttl + '\n' + out2parstr(out))

In [ ]:
for out in outs:
    del out['params']['params']

In [ ]:
p = out['params'].copy()
print('params' in p)
p.keys()

In [ ]:
coln_error_ = 'error_deg'
pertr = ['states', 'y_pred',  'adaptation_rate'] #'optres',
rows = []
for (dfn,subj),outs in dfnamesubj2res_Tan.items():
    qs = 'subject == @subj'    
    for outi,out in enumerate(outs):
        print(subj, outi, out['fit_trials'])
        p = out['params'].copy()
        assert 'params' not in p
        for k in out.keys():
            if k in ['params','optres']:
                continue
            elif k not in pertr:
                p[k] = out[k]
        p['fit_success'] = out['optres'].success
        p['ind'] = outi
        p['subject'] = subj

        assert 'params' not in p
        pp = {}
        for k in p.keys():
            pp[k] = [p[k]] * len(out['states'])
        assert 'params' not in pp
        for k in pertr:
            pp[k] = out[k]
        assert 'params' not in pp
        pp['trial_index'] = np.arange(len(out['states']))
        
        dfos = dfcc_all_NIH.query(qs) 
        if p['fit_trials'] == 'stable':
            dfos = dfos.query('environment == 0')
        elif p['fit_trials'] == 'random':
            dfos = dfos.query('environment == 1')
            
        print(len( out['states']), len(dfos) )
            
        pp['error'] = dfos[coln_error_].values
        pp['perturbation'] = dfos['perturbation'].values
        pp['mismatch'] = out['y_pred'] + dfos[coln_error_].values
        pp['environment'] = dfos['environment'].values
        
        subdf = pd.DataFrame(pp)
        subdf['mismatch_0'] = np.sqrt( out['mismatch_var'] ) / len(subdf)
        st = subdf.query('environment == 0')
        if len(st):
            subdf['mismatch_stable'] = np.sqrt( (st['mismatch']**2).sum() ) / len(st)
        else:
            subdf['mismatch_stable'] = np.nan
            
        rnd = subdf.query('environment == 1')
        if len(rnd):
            subdf['mismatch_random'] = np.sqrt( (rnd['mismatch']**2).sum() ) / len(rnd)
        else:
            subdf['mismatch_random'] = np.nan

        subdf['mismatch_all'] = np.sqrt( (subdf['mismatch']**2).sum() ) / len(subdf)
        rows += [subdf]
        
df = pd.concat(rows, ignore_index=1)

In [ ]:
gc. collect()

In [ ]:
#pp['fit_trials']

In [ ]:
len(out['y_pred']), len(out['states'])

In [ ]:
p['fit_trials']

In [ ]:
len(dfos)

In [ ]:
df.columns

In [ ]:
out['params']

In [ ]:
out.keys()

In [ ]:
coln_error_

In [ ]:
dfsrt = df.sort_values('mismatch_0')
dfsrt['powdiff'] = dfsrt['stats_powers'].apply(lambda x: x[0] + x[1], 1)
dftest_pre = dfsrt.query('trial_index == 82')

In [ ]:
import seaborn as sns

In [ ]:
import pingouin as pg
pg.corr(dfsrt['mismatch_stable'],dfsrt['powdiff'])

In [ ]:
pg.corr(dfsrt['mismatch_all'],dfsrt['powdiff'])

In [ ]:
pg.corr(dfsrt['mismatch_stable'],dfsrt['mismatch_all'])

In [ ]:
with warnings.catch_warnings(record=True) as w:
    #warnings.filterwarnings('ignore',category=RuntimeWarning)
    sns.pairplot(dfsrt, vars=['powdiff','mismatch_stable','mismatch_random'], hue='reg', corner=True )

In [ ]:
dftest = dftest_pre.tail(20)
dftest[cols]

In [ ]:
with warnings.catch_warnings(record=True) as w:
    warnings.filterwarnings('ignore',category=FutureWarning)
    sns.catplot(data=dftest_pre, kind='violin', y='mismatch_all', x='powdiff',row='reg',hue='fit_success',
            col='fit_trials',
           split=True)

In [ ]:
#'perturbation'
dftest = dfsrt.query('trial_index == 82').head(60)
cols = ['ind','error','y_pred','A','c','fit_success','reg',
    'use_true_error','use_true_error_stats','stats_powers','NbPreviousTrials','X0','fit_trials',
        'mismatch_all','mismatch_stable']
dftest[cols]

In [ ]:
dfsrt.columns

In [ ]:
dfsrt['error_abs'] = dfsrt['error'].abs()
dfsrt['states_abs'] = dfsrt['states'].abs()
dfsrt['y_pred_abs'] = dfsrt['y_pred'].abs()

In [ ]:
dfsrt['mispred'] = (dfsrt['y_pred'] - dfsrt['error']).abs()

In [ ]:
dfsrt['stats_powers'].unique(), dfsrt['reg'].unique()

In [ ]:
stp = (0,-1)
regs = [0., 6.]
#use_true_error == True and 
df_ = dfsrt.query('use_true_error_stats == True and fit_trials == "all"'
                  ' and reg in @regs') # ' and reg == 0. and stats_powers == @stp 
df_.groupby(['trial_index']).size().max()

In [ ]:
df__ = dfsrt.query('use_true_error_stats == True and fit_trials == "all"'
                  ' and reg == 0. and stats_powers == @stp' )
df__.groupby(['trial_index']).size().max()

In [ ]:
df__.query('trial_index == 81').groupby(['use_true_error'])['mismatch_all'].describe()

In [ ]:
df__.query('trial_index == 81').groupby(['use_true_error'])['mispred'].describe()

In [ ]:
df__.query('trial_index == 81').groupby(['use_true_error'])['y_pred_abs'].describe()

In [ ]:
# (in implem if use true error =1 then yp =  error[t-1]) 
# if use true error = 0 AND stat power for mean == 0 
#   => mismatch grow in first pertstage (both before and after random)
#     for stat powers (1,-2) it is actually okay 
# well, this is not stable across subjects. Like above was only for subject0

# using true of false error stats makes diff only if use true error =0 
# and only for some stat powers combin (1,-2)

In [ ]:
with warnings.catch_warnings(record=True) as w:
    warnings.filterwarnings('ignore',category=FutureWarning)
    sns.pairplot(dftest_pre.query('use_true_error == False'),
                 vars=['c','mismatch_stable','mismatch_all'], hue='stats_powers')

In [ ]:
with warnings.catch_warnings(record=True) as w:
    warnings.filterwarnings('ignore',category=FutureWarning)
    #df__ = df_.query('subject == @subjects_NIH[12]')
    df__ = df_.groupby(['environment','stats_powers','reg','use_true_error','subject'])['mispred'].mean().reset_index()
    fg = sns.catplot(df__, kind='violin', y='mispred',
                x='environment',col='stats_powers', row='reg', errorbar='sd', hue='use_true_error',
                    split=True, hue_order=[0,1])
    for ax in fg.axes.flat:
        ax.yaxis.grid(True)

In [ ]:
df_ = dfsrt.query('fit_trials == "all"'
                  ' and reg in @regs and use_true_error == True') # ' and reg == 0. and stats_powers == @stp 
df_.groupby(['trial_index']).size().max()
with warnings.catch_warnings(record=True) as w:
    warnings.filterwarnings('ignore',category=FutureWarning)
    #df__ = df_.query('subject == @subjects_NIH[12]')
    df__ = df_.groupby(['environment','stats_powers','reg','use_true_error_stats','subject'])['mispred'].mean().reset_index()
    fg = sns.catplot(df__, kind='violin', y='mispred',
                x='environment',col='stats_powers', row='reg', errorbar='sd', hue='use_true_error_stats',
                    split=True, hue_order=[0,1])
    for ax in fg.axes.flat:
        ax.yaxis.grid(True)

In [ ]:
stp = (0,-1)
regs = [0.]
df_ = dfsrt.query('fit_trials == "all"'
                  ' and reg in @regs and stats_powers == @stp')
df_.groupby(['trial_index']).size().max()
with warnings.catch_warnings(record=True) as w:
    warnings.filterwarnings('ignore',category=FutureWarning)
    #df__ = df_.query('subject == @subjects_NIH[12]')
    df__ = df_.groupby(['environment','stats_powers','reg','use_true_error_stats','use_true_error',
                        'fit_success','subject'])['mispred'].mean().reset_index()
    fg = sns.catplot(df__, kind='violin', y='mispred',
                x='environment',col='use_true_error_stats', row='use_true_error', errorbar='sd', 
                     hue='fit_success',
                    split=True, hue_order=[0,1])
    for ax in fg.axes.flat:
        ax.yaxis.grid(True)

In [ ]:
#use_true_error_stats == True
with warnings.catch_warnings(record=True) as w:
    warnings.filterwarnings('ignore',category=FutureWarning)
    fg = sns.relplot(df_, kind='line', y='mispred',
                x='trial_index',col='stats_powers', row='reg', errorbar='sd', hue='use_true_error')
#    fg.grid(True)
    for ax in fg.axes.flat:
        ax.yaxis.grid(True)

In [ ]:
with warnings.catch_warnings(record=True) as w:
    warnings.filterwarnings('ignore',category=FutureWarning)
    fg = sns.relplot(df_.query('subject == @subjects_NIH[2]'), kind='line', y='mispred',
                x='trial_index',col='stats_powers', row='reg', errorbar='sd', hue='use_true_error')

In [ ]:
good_inds = dftest['ind']

In [ ]:
good_inds

In [ ]:
out['optres']['success'],
                    out['stats_powers'], 
                    out['online_feedback'],out['use_true_error'], 
                    out['use_true_error_stats'], out['reg'], out['minfval']out['params']

In [ ]:
out['optres']['success'],
                    out['stats_powers'], 
                    out['online_feedback'],out['use_true_error'], 
                    out['use_true_error_stats'], out['reg'], out['minfval']

In [ ]:
dfnamesubj2res_Tan.items()

In [ ]:
# plot many
for (dfn,subj),outs in dfnamesubj2res_Tan.items():
    #for out in outs:
    for ind,out in np.array(list(enumerate(outs ) ))[good_inds]:
        if 'adaptation_rate' in out:
            adaptationRate = out['adaptation_rate']
            output = out['y_pred']
        else:
            adaptationRate = None
            output = out['output']
        print(ind,subj)
        dfos = dfcc_all_NIH.query('subject == @subj')

        coln_error_ = 'error'
        error   = dfos[coln_error_].to_numpy() * 180 / np.pi
        perturb = dfos['perturbation'].values
        adaptationRate = out['adaptation_rate']

        plt.figure(figsize=(15,3)) # Use matplotlib.pyplot to plot figures
        plt.title('Subject ' + dfos['subject'].values[0]) # Use string concatenation to make titles
        plt.plot(error, linewidth=2, label='Observed error') # Use label argument to add legends
        plt.plot(out['y_pred'], linewidth=2, label='Model prediction',alpha=0.8)
        plt.plot(out['states'], label='states', c='purple')
        plt.ylabel('Angle (°)')
        plt.xlabel('Trials')
        plt.plot(perturb, label='pert', color='red', ls=':')
        plt.legend() # Call legend function to show legends
        #plt.twinx() # Use twinx function to create a second y-axis
        if adaptationRate is not None:
            plt.plot(30 + 50*adaptationRate, linewidth=1, label='Adaptation rate', color='green')            
            plt.legend() # Call legend function to show legends
        plt.ylim(-40,60)
        plt.grid(True)

        ttl = 'Subject {}, optres={}, pows={} onlfb={}, truerr={}, trueerrst={}, reg={}, minf={:.3f} fit_trials={}'.format( 
                dfos['subject'].values[0][:5],
                out['optres']['success'],
                    out['stats_powers'], 
                    out['online_feedback'],out['use_true_error'], 
                    out['use_true_error_stats'], out['reg'], out['minfval'], out['fit_trials'] )
        plt.title(ttl + '\n' + out2parstr(out))
        plt.show() # Call show function to display the figure
        #break

In [ ]:
gc.collect()

In [ ]:
if len(dfnamesubj2res_Tan) == 20:    
    fnf = pjoin(path_data_cc,'dfnamesubj2res_Tan_many.npz')
    print(len(dfnamesubj2res_Tan), fnf)
    np.savez(fnf, dfnamesubj2res_Tan)

In [ ]:
out['params']

In [ ]:
if len(dfnamesubj2res_Tan) == 20:    
    fnf = pjoin(path_data_cc,'dfnamesubj2res_Tan_fboffl.npz')
    print(len(dfnamesubj2res_Tan), fnf)
    np.savez(fnf, dfnamesubj2res_Tan)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
COLOR = 'black'
import matplotlib as mpl
import seaborn as sns
mpl.rcParams['text.color'] = COLOR
mpl.rcParams['axes.labelcolor'] = COLOR
mpl.rcParams['xtick.color'] = COLOR
mpl.rcParams['ytick.color'] = COLOR


sns.set(rc={'axes.facecolor':'white', 'figure.facecolor':'white',
           'grid.color':'grey', 'grid.alpha':0.5})

tgtc = ['b','g','r','brown']

In [ ]:
nurns = 4
outd = {}
dfos = df.query('subject == @subj')
outn = 'all_off'; out = fitDiedrischenModel(dfos, coln_error_,
                                 nruns=nruns, online_feedback = False); outd[outn] = out
outn = 'all_on'; out = fitDiedrischenModel(dfos, coln_error_,
                                 nruns=nruns, online_feedback = True); outd[outn] = out
qsst = 'environment == 0'
outn = 'stable_off'; out = fitDiedrischenModel(dfos.query(qsst), coln_error_,
                                 nruns=nruns, online_feedback = False); outd[outn] = out
outn = 'stable_on'; out = fitDiedrischenModel(dfos.query(qsst), coln_error_,
                                 nruns=nruns, online_feedback = True); outd[outn] = out

qsst = 'environment == 1'
outn = 'random_off'; out = fitDiedrischenModel(dfos.query(qsst), coln_error_,
                                 nruns=nruns, online_feedback = False); outd[outn] = out
outn = 'random_on'; out = fitDiedrischenModel(dfos.query(qsst), coln_error_,
                                 nruns=nruns, online_feedback = True); outd[outn] = out
#output_fboff = out_fboff['output']
#output_fbon  = out_fbon['output']
# -output + perturb

In [ ]:
for outn,out in outd.items():
    print(outn, dict(zip(['A','B','X0'],out['params'])) )

In [ ]:
from taumodels.state_space_fit_toolbox.state_space_fit import simulate_deterministic_state_space2
inds_st = np.hstack([np.arange(192),np.arange(192*2,192*3)])
inds_rnd = np.hstack([np.arange(192,192*2),np.arange(192*3,192*4)])

perturb = df.query('subject == @subj')['perturbation'].values
perturb_s = df.query('subject == @subj')['perturbation'].shift(1).values; perturb_s[0] = 0
behavior = perturb  - error
error   = df.query('subject == @subj')[coln_error_].values
for pref in ['random','all','stable']:
    #dif = outd[f'{pref}_off']['output'] - outd[f'{pref}_on']['output']
    y_pred_d={}
    for fb in ['on','off']:            
        out = outd[f'{pref}_{fb}']
        S = out['params']
        #print(S)
        if fb == 'on':
            y_pred,state = simulate_deterministic_state_space(S,perturb,behavior,'norm')
        else:
            y_pred,state,errp = simulate_deterministic_state_space2(S,perturb,behavior,'norm')
        y_pred_d[fb]  = y_pred
        err_pred = perturb - y_pred # y_pred - (perturb - err)
        
        plt.figure(figsize=(10,3))
        plt.plot(error, linewidth=2, label='Observed error') 
        #plt.plot(-output + perturb, linewidth=2, label='Model prediction',alpha=0.8)
        plt.plot(err_pred, linewidth=2, label='Model prediction',alpha=0.8)
        #plt.plot(perturb-error, linewidth=2, label='pert-err') 

        plt.ylabel('Angle (°)')
        plt.xlabel('Trials')
        plt.plot(perturb, label='pert', color='pink', ls=':')
        plt.legend() # Call legend function to show legends
        #plt.twinx() # Use twinx function to create a second y-axis
        plt.ylim(-40,60)
        plt.grid(True)
        discr = np.abs(err_pred - error)
        discr_mean = np.mean(discr)
        discr_mean_st = np.mean(discr[inds_st])
        discr_mean_rnd = np.mean(discr[inds_rnd])
        plt.title(f'{pref}_{fb}  discr={discr_mean:.3f} st={discr_mean_st:.3f} rnd={discr_mean_rnd:.3f}')
        plt.show()
    dif = np.abs(y_pred_d['on'] - y_pred_d['off'])
    print(pref,S,   np.std(dif[inds_st]), np.std(dif[inds_rnd]) )
    #err_pred = y_pred - behavior
    #if 
    #print( np.std( dif[inds_st  ]), np.std( dif[inds_rnd  ]) )

In [ ]:
sl = slice(0,200)
simulate_deterministic_state_space2(S,perturb[sl],behavior[sl],'norm',1);

In [ ]:
plt.plot(perturb, ls='--', c='pink')
plt.plot(error, label='obs err')
plt.plot(y_pred, label='state')
plt.plot(perturb-y_pred, label='pert-state')
plt.plot(perturb-error, label='beh=pert-err')
#plt.plot(errp, label='errP')
plt.axhline(y=0, ls=':', c='r')
plt.axvline(x=192, ls=':', c='r')
plt.legend()
plt.xlim(30,250)

In [ ]:
out_fboff.keys()

In [ ]:
np.abs(output_fboff - output_fbon).max() / np.std(output_fboff)

In [ ]:
out_fboff['noises_var']

In [ ]:
df['subject']

# Died

In [ ]:
dfnamesubj2res_Died = {}
nruns = 6
#nsubj = 1
nsubj = 2
#fit_trials = 'stable'
fit_trials = 'all'
#for dfname,df in zip(['NIH', 'cc'], [dfcc_all_NIH,dfcc_all]):
for dfname,df in zip(['NIH'], [dfcc_all_NIH]):
    for subj in df.subject.unique()[:nsubj]:
        print('   subj == ',subj)
        qs = 'subject == @subj'
        t0 = time.time()
        if dfname == 'NIH':
            coln_error_ = 'error_deg'
            if fit_trials == 'stable':
                qs += ' and environment == 0'
        else:         
            qs += ' and trial_index >= @numtrain'            
            coln_error_ = coln_error
        dfos = df.query(qs)   

        t0 = time.time()
        out_cursubj = fitDiedrischenModel(dfos, coln_error_,
                             nruns=nruns, online_feedback = False)
        outD = out_cursubj
        t1 = time.time()
        print(f'Fit of Diedrischen model took {t1-t0:.2f} sec')
        #dict_keys(['output', 'params', 'asymptote', 'noises_var', 'states', 'performances'])
        
        dfnamesubj2res_Died[(dfname,subj)] = out_cursubj
        
        perturb = dfos['perturbation'].values
        #error   = dfos[coln_error_].to_numpy() 
        #output = out_cursubj['output']
        
        error =df.query('subject == @subj')[coln_error_].values
        perturb = df.query('subject == @subj')['perturbation'].shift(1).values
        behavior = perturb  - error
        #perturb = df.query('subject == @subj')['perturbation'].shift(1).values
        from taumodels.state_space_fit_toolbox.state_space_fit import simulate_deterministic_state_space
        S = out_cursubj['params']
        y_pred,state = simulate_deterministic_state_space(S,perturb,behavior,'norm')
        err_pred = perturb - y_pred 
                
        plt.figure(figsize=(15,3)) # Use matplotlib.pyplot to plot figures
        plt.title('Subject ' + dfos['subject'].values[0])         
        plt.plot(error, linewidth=2, label='Observed error') 
        #plt.plot(-output + perturb, linewidth=2, label='Model prediction',alpha=0.8)
        plt.plot(err_pred, linewidth=2, label='Model prediction',alpha=0.8)
        #plt.plot(perturb-error, linewidth=2, label='pert-err') 

        plt.ylabel('Angle (°)')
        plt.xlabel('Trials')
        plt.plot(perturb, label='pert', color='pink', ls=':')
        
        plt.plot(y_pred,label='state')
        plt.legend() # Call legend function to show legends
        #plt.twinx() # Use twinx function to create a second y-axis
        plt.ylim(-40,60)
        plt.grid(True)
        plt.show() # Call show function to display
        #break

In [ ]:
#error

In [ ]:
if len(dfnamesubj2res_Died) == 20:
    fnf = pjoin(path_data_cc,'dfnamesubj2res_Died_fboffl.npz')
    print(len(dfnamesubj2res_Died), fnf)
    np.savez(fnf, dfnamesubj2res_Died)

In [ ]:
# print('Start parallel over nbTrials and startIdx')
# from joblib import Parallel, delayed
# nruns = 5
# plr = Parallel(n_jobs=n_jobs, backend='multiprocessing',
#                )(delayed(calcH_min)(-perturb, error, EC_mask) for i in np.arange(nruns))


# Herz

In [ ]:
# async not parallel
timeout = 12
from taumodels.models import simMoE, calcH_min
from taumodels.models import getBestHerzPar
def fitModelHerz_simple(perturb, error, EC_mask, nruns = 20, 
                 timeout=12, reg=0):
    plr = []
    for i in range(nruns*10):
        # Create a queue to store the result
        queue = mp.Queue()
        # Create a process that runs the optimize function and puts the result in the queue
        p = mp.Process(target=lambda q, x: 
            q.put(_opt(x)), 
                args=(queue, (perturb, error, EC_mask,reg)))
        p.start()
        p.join(timeout)

        # Check if the process is still alive
        if p.is_alive():
            p.terminate()
            p.join()
            print("------ Terminating optimization: time limit reached")
            # Return None
            res = None,None,None
        else:
            # Get the result from the queue
            res = queue.get()

        if isSuccess(res):
            plr += [res]
        if len(plr) >= nruns:
            break
        else:
            print(len(plr))
    return plr

def isSuccess(res, check_q = False):
    (minres_H, pard, seed) = res
    #print('isSuccess: pard={} minres={}'.format(pard,minres_H))
    if pard is None:
        return False
    r = simMoE(-perturb, EC_mask=EC_mask, **pard)
    motor_output,error_pred2,ws,err_sens2, gaussian_centers,gaussian_variances = r 
    
    if check_q:
        q = np.quantile(err_sens2, 0.9) 
    else:
        q = 0
    if minres_H.success and q < 0.98:
        return True
    return False

import multiprocessing as mp
def _opt(_x):
    perturb, error, EC_mask, pre_break_duration, reg = _x
    s = int(np.ceil(time.time() * 1e4) ) % int( 1e5 )
    try:        
        print('Starting opt with seed = ',s)
        np.random.seed( s )
        minres_H, pard = calcH_min(-perturb, error, EC_mask, pre_break_duration,
                                  reg=reg, timeout = timeout)
    except ValueError as e:
        raise e
        print(e)
        pard = None
        minres_H = None
    return (minres_H, pard, s)

In [ ]:
import sys,os
import numpy as np
sys.path.append(os.path.expandvars('$CODE_MEMORY_ERRORS'))
from taumodels.models import simMoE, calcH_min
perturb = np.array([0,0,30])
error = perturb
EC_mask = np.array([0,0,0])
pre_break_duration = np.array([0,0,0])
reg = 0

#%debug
#_opt((perturb, error, EC_mask, pre_break_duration, reg))

calcH_min(-perturb, error, EC_mask, pre_break_duration,
                                  reg=reg, timeout = 12)

In [ ]:
np.max(pre_break_duration)

In [ ]:
import gc; gc.collect()

In [ ]:
dfcc_all_NIH.query('pre_break_duration > 0')['pre_break_duration']

In [ ]:
dfnamesubj2res_Herz = {}

In [ ]:
nruns = 20
max_attempts = 9
timeout = 50
n_jobs=25

nruns = 12

# max_attempts = 25
# timeout = 10
# nruns = 5
# reg = 1

addonly = 1
# nruns = 4
# max_attempts = 4 # reattemptts actually
# timeout = 7
print(len(dfnamesubj2res_Herz ))
#for dfname,df in zip(['NIH', 'cc'], [dfcc_all_NIH,dfcc_all]):
for dfname,df in zip(['NIH'], [dfcc_all_NIH]):
    for subj in df.subject.unique()[::-1]:
        if addonly and (dfname,subj) in dfnamesubj2res_Herz:
            if len(dfnamesubj2res_Herz[(dfname,subj)]) > 1:
                continue
        print('-------------------')
        print('-------------------')
        print('########### ',dfname, '   subj == ',subj)
        print('-------------------')
        print('-------------------')
        qs = 'subject == @subj'
        t0 = time.time()
        if dfname == 'NIH':
            coln_error_ = 'error_deg'
        else:         
            qs += ' and trial_index >= @numtrain'            
            coln_error_ = coln_error
        
        clear_outlier = 120
        dfos = df.query(qs).copy()
        inds = dfos.query(
            f'{coln_error_}.abs() > {clear_outlier}').index
        dfos.loc[inds,coln_error_] = np.nan
        
        
        error = dfos[coln_error_].values
        perturb = dfos['perturbation'].values
        
        perturb[perturb > 180] = perturb[perturb > 180] - 360
        
        if dfname == 'NIH':
            EC_mask = np.array( [0] * len(dfos) )
        else:
            EC_mask = (dfos['trial_type'] == "error_clamp").values
        
        pre_break_duration = dfos['pre_break_duration'].values
            
        #out_cursubj = fitModelHerz(perturb, error, EC_mask, nruns = nruns, 
        #                    max_attempts = max_attempts, timeout=timeout,
        #                          n_jobs=n_jobs)
        #out_cursubj = fitModelHerz_simple(perturb, 
        #        error, EC_mask, nruns = nruns, 
        #        timeout=timeout, reg=reg)
        plr = []
        dfnamesubj2res_Herz[(dfname,subj)]  = []
        for i in range(nruns*max_attempts):
            # Create a queue to store the result
            queue = mp.Queue()
            # Create a process that runs the optimize function and puts the result in the queue
            p = mp.Process(target=lambda q, x: 
                q.put(_opt(x)), 
                    args=(queue, (perturb, error, EC_mask, pre_break_duration, reg)))
            p.start()
            p.join(timeout)

            # Check if the process is still alive
            if p.is_alive():
                p.terminate()
                p.join()
                print(f"------ {subj} Terminating optimization: time limit reached")                
                res = None,None,None
            else:
                res = queue.get()

            if isSuccess(res):
                dfnamesubj2res_Herz[(dfname,subj)].append(res)
                plr.append(res)
                print('-------------------')
                print('get output but with success!'
                      ' new len(plr) = {len(plr)} ( of needed {nruns})')
                print('-------------------')
            else:
                print('-------------------')
                print('get output but w/ot success')
                print('-------------------')
            
            if len(plr) >= nruns:
                print(f'Finishing for {subj} after {i+1} runs')
                break
            else:
                print(f'---- {subj} Len plr = ',len(plr))
                
        dfnamesubj2res_Herz[(dfname,subj)] = plr
#         break
#     break
        pard0 = getBestHerzPar(plr)        
        pard = pard0.copy()
        if 'fun' in pard:
            del pard['fun']
            del pard['nit']
            del pard['nfev']
            del pard['seed']
        r = calcH_orig(-perturb,EC_mask=EC_mask, **pard)
        (motor_output,error_pred2,ws,err_sens2, 
            gaussian_centers,gaussian_variances) = r 

        plt.figure(figsize=(12,3))
        plt.plot(error,label='true')
        plt.plot(error_pred2,label='pred')
        plt.plot(perturb,label='pert', c='r',alpha=0.7, ls=':')
        plt.plot(30 + err_sens2*20,label='ES')
        plt.axhline(30, ls=':', c='k')
        plt.axhline(30+20, ls=':', c='k')
        plt.grid(True)
        plt.title('fval = {:.3f}; alpha={:.3f} alpha_w={:.3f}'.\
                  format(pard0['fun'], pard['alpha'], pard['alpha_w']))
        plt.legend()
        plt.show()

In [ ]:
bads = []
for k,v in dfnamesubj2res_Herz.items():
    if len(v) == 0:
        bads += [k]
print(bads)
for k in bads:
    del dfnamesubj2res_Herz[k]

In [ ]:
# subj014 does not fit Herz

In [ ]:
len(plr), len(dfnamesubj2res_Herz)

In [ ]:
dfnamesubj2res_Herz.keys()

In [ ]:
from pprint import pprint

In [ ]:
out_cursubj

In [ ]:
out_cursubj = plr

In [ ]:
        pard0 = getBestHerzPar(plr)        
        pard = pard0.copy()
        if 'fun' in pard:
            del pard['fun']
            del pard['nit']
            del pard['nfev']
            del pard['seed']
        r = calcH_orig(-perturb,EC_mask=EC_mask, **pard)
        (motor_output,error_pred2,ws,err_sens2, 
            gaussian_centers,gaussian_variances) = r 

        plt.figure(figsize=(12,3))
        plt.plot(error,label='true')
        plt.plot(error_pred2,label='pred')
        plt.plot(perturb,label='pert', c='r',alpha=0.7, ls=':')
        plt.plot(30 + err_sens2*20,label='ES')
        plt.axhline(30, ls=':', c='k')
        plt.axhline(30+20, ls=':', c='k')
        plt.grid(True)
        plt.title('fval = {:.3f}; alpha={:.3f} alpha_w={:.3f}'.\
                  format(pard0['fun'], pard['alpha'], pard['alpha_w']))
        plt.legend()

In [ ]:
if len(dfnamesubj2res_Herz):
    fnf = pjoin(path_data_cc,'dfnamesubj2res_Herz.npz')
    print(len(dfnamesubj2res_Herz), fnf)
    np.savez(fnf, dfnamesubj2res_Herz)

In [ ]:
# using async IO to handle optim prone to hang

import time
def fitModelHerz(perturb, error, EC_mask, nruns = 20, 
                 max_attempts = 5, timeout=30, n_jobs=8):
    attempts_per_run_left = np.array( nruns*[max_attempts] )
    timouts_per_run_left = np.array( nruns*[max_attempts] )  # maybe redundant
    success_per_run = np.array( nruns * [False] )

    plrpre = []
    plr2 = []
    pool = mp.Pool(min(n_jobs, nruns) )
    for i in range(nruns):
        result = pool.apply_async(_opt, args=((perturb, error, EC_mask),))
        plrpre.append(result)

    while (not np.all(success_per_run) ) and np.sum( attempts_per_run_left ) > 0: 
        # Assume that all the optimizations succeeded unless proven otherwise success = True 
        # Loop over the list of results and their corresponding initial guesses 
        for i, result in enumerate(plrpre): 
            # if timout it is not always (never?) ready
            if result.ready(): # Use the successful method of the result object to check 
                if (not result.successful()) or (not success_per_run[i]): # If it has failed, set the success flag to False 
                    success_per_run[i] = False # Print a message
                    if attempts_per_run_left[i] == 0:
                        continue
                    print(f"Optimization {i} failed, restarting") 
                    result = pool.apply_async(_opt, args=((perturb, error, EC_mask),))
                    attempts_per_run_left[i] -= 1
                    plrpre[i] = result 
            else: 
                success_per_run[i] = False

        # If all the optimizations have succeeded, break out of the loop
        npoor1 = np.sum( attempts_per_run_left[~success_per_run] )
        npoor2 = np.sum( timouts_per_run_left[~success_per_run] )
        if np.all(success_per_run) or npoor1 == 0 or npoor2 == 0:
            print('Terminate everything')
            pool.terminate()
            pool.join()
            time.sleep(5)
            break 

        for i, result in enumerate(plrpre):
            if success_per_run[i]:
                continue
            try:
                res = result.get(timeout=30)
                succ = isSuccess(res)
                print('Wrk {} check Success. Ready={}, succ0={}, succ1={}; len(plr2) = {},\n attempts={}, still fail={}'.\
                      format(i,result.ready(),result.successful(), succ, len(plr2) ,
                      attempts_per_run_left[~success_per_run], np.where(~success_per_run)[0] ) )
                if succ:
                    plr2 += [res]
                    success_per_run[i] = True
            except mp.TimeoutError: # Handle any exceptions raised by the get method
                # Print a message
                timouts_per_run_left[i] -= 1            
                npoor = np.sum( timouts_per_run_left[~success_per_run] )
                print(f"Terminating {i}: time limit reached ready={result.ready()}, sum tl={npoor}")
                
        npoor = np.sum( timouts_per_run_left[~success_per_run] )
        if npoor == 0:
            # Terminate the pool
            print('Terminate everything 2')
            pool.terminate()
            pool.join()
            time.sleep(5)
            break

    if np.all(success_per_run):
        print("All optimizations succeeded")
    else:
        print("Maximum number of attempts reached: succesed {} of {}".format( np.sum(success_per_run), 
                                                                             nruns))
        pool.terminate()
        pool.join()
    return plr2

In [ ]:
best_par0 = {'initial_sensitivity': 0.3661433958521835, 'gaussian_variance': 3.850651446260379, 'eta': 0.06792935414425082, 'alpha': 0.694112481631201, 'execution_noise_variance': 2.369905698380425, 'sensory_noise_variance': 0.40409512579074175, 'alpha_w': 0.9385030600766864, 'fun': 64.69090744102397, 'nit': 5.0, 'nfev': 8.0}
best_par1 = {'initial_sensitivity': 0.33031854635212193, 'gaussian_variance': 3.5238276546075986, 'eta': 0.14726827889543936, 'alpha': 0.848256765609661, 'execution_noise_variance': 1.2807575561680065, 'sensory_noise_variance': 1.4880866262162167, 'alpha_w': 0.4708160119528675, 'fun': 65.65002206900775, 'nit': 6.0, 'nfev': 26.0}

In [ ]:
plr2 = list(dfnamesubj2res_Herz.values())[0]

In [ ]:
import pandas as pd
rows = []
for tpl in plr2:
    d = tpl[1].copy()
    d['fun'] = tpl[0].fun
    d['nit'] = tpl[0].nit
    d['nfev'] = tpl[0].nfev
    d['seed'] = tpl[2]
    rows += [d]
dfrespar = pd.DataFrame(rows)#.describe()
if len(dfrespar):
    display( dfrespar.describe() )

    ind =  dfrespar['fun'].idxmin()
    print(ind)
    best_par = dfrespar.loc[ind].to_dict()
    print(best_par)

In [ ]:
dfrespar

In [ ]:
len( plr2 )

In [ ]:
pard

In [ ]:
#plr = [(minres_H, pard)]

In [ ]:
plt.hist(err_sens2)

In [ ]:
q = np.quantile(err_sens2, 0.9) 
print(q)
#-  0.8

In [ ]:
pard

In [ ]:
EC_mask = (dfos['trial_type'] == "error_clamp").values
#EC_mask = None
#%debug
error = dfos[coln_error].values
perturb = dfos['perturbation'].values
r = calcH_orig(-perturb,EC_mask=EC_mask)
motor_output,error_pred,ws,err_sens, gaussian_centers,gaussian_variances = r 

In [ ]:
plt.figure(figsize=(12,3))
plt.plot(error,label='true')
plt.plot(error_pred,label='pred')
plt.plot(perturb,label='pert', c='r',alpha=0.7, ls=':')
plt.plot(30 + err_sens*20,label='ES')
plt.axhline(30, ls=':', c='k')
plt.axhline(30+20, ls=':', c='k')
plt.grid(True)
plt.legend()

for tpl in plr2:
    minres,pard = tpl
    if 'fun' in pard:
        del pard['fun']
    r = calcH_orig(-perturb,EC_mask=EC_mask, **pard)
    motor_output,error_pred2,ws,err_sens2, gaussian_centers,gaussian_variances = r 

    plt.figure(figsize=(12,3))
    plt.plot(error,label='true')
    plt.plot(error_pred2,label='pred')
    plt.plot(perturb,label='pert', c='r',alpha=0.7, ls=':')
    plt.plot(30 + err_sens2*20,label='ES')
    plt.axhline(30, ls=':', c='k')
    plt.axhline(30+20, ls=':', c='k')
    plt.grid(True)
    plt.title('fval = {:.3f}; alpha={:.3f} alpha_w={:.3f}'.format(minres.fun, pard['alpha'], pard['alpha_w']))
    plt.legend()

In [ ]:
ind

# plot

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
#for subj in subj2out_Tan:
for out in [outT, outD]:
    error   = dfos[coln_error].to_numpy()
    perturb = dfos['perturbation'].to_numpy()    
    #out = subj2out_Tan[subj ]
    adaptationRate = None
    if 'adaptation_rate' in out:
        adaptationRate = out['adaptation_rate']
    output = out['y_pred']
    plt.figure(figsize=(15,3)) # Use matplotlib.pyplot to plot figures
    plt.title('Subject ' + dfos['subject'].values[0]) # Use string concatenation to make titles
    plt.plot(error, linewidth=2, label='Observed error') # Use label argument to add legends
    plt.plot(output, linewidth=2, label='Model prediction',alpha=0.8)
    plt.ylabel('Angle (°)')
    plt.xlabel('Trials')
    plt.plot(perturb, label='pert', color='red', ls=':')
    plt.legend() # Call legend function to show legends
    #plt.twinx() # Use twinx function to create a second y-axis
    if adaptationRate is not None:
        plt.plot(30 + 50*adaptationRate, linewidth=1, label='Adaptation rate', color='green') # Use color argument to change the line color
        plt.legend() # Call legend function to show legends
    plt.ylim(-40,60)
    plt.grid(True)
    plt.show() # Call show function to display the figure
